In [21]:
import pandas as pd
from datetime import datetime
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

DATA_FILE = 'data/list_of_cars2.csv'

def preprocess(data_file):
    df = pd.read_csv(data_file,header=None, names=['Brand', 'Body', 'Model', 'Year', 'Price', 'HP', 'Torque', 'Acc', 'Braking', 'Road Holding', 'FE-MPG-CMB', 'FE-MPG-CTY', 'FE-MPG-HWY', 'FE-L100K-CMB', 'FE-L100K-CTY', 'FE-L100K-HWY'])
    df['HP'] = pd.to_numeric(df['HP'].apply(lambda x: str(x)[:3].strip()), errors='coerce').astype(np.float64).dropna()
    df['Torque'] = pd.to_numeric(df['Torque'].apply(lambda x: str(x)[:3].strip()), errors='coerce').astype(float).dropna()
    df['Braking'] = pd.to_numeric(df['Braking'].apply(lambda x: str(x)[:3].strip()), errors='coerce').astype(float).dropna()
    df['Road Holding'] = pd.to_numeric(df['Road Holding'].apply(lambda x: str(x)[:4].strip()), errors='coerce').astype(float).dropna()
    df['Fuel Economy'] = pd.to_numeric(df['FE-MPG-CMB'].apply(lambda x: str(x).split("/")[0].replace("NA","0")), errors='coerce').astype(float).dropna()

    return df.dropna(thresh=3)

def calculate_score(row, average_score, max_score, attribute):
    
    final_score = 0
    
    if (attribute != 'Braking' and row[attribute] >= max_score) or (attribute == 'Braking' and row[attribute] <= max_score):
        return 5
    else: 
        if attribute == 'Braking':
            #Calculating braking is inverted, lower is better thus the if condition is different
            if row[attribute] < average_score: 
                calculate_difference= (max_score - row[attribute])/(max_score-average_score)
                final_score = 5 - 2.5 * calculate_difference
            else: 
                final_score = (average_score/row[attribute]) * 2.5
        elif row[attribute] > average_score:
            calculate_difference= (row[attribute] - average_score)/(max_score - average_score)
            final_score = 2.5 + 2.5 * calculate_difference
        else:
            final_score = (row[attribute]/average_score * 2.5)
            
    return final_score

data = preprocess(DATA_FILE)

avg_HP = data['HP'].mean()
max_HP = data['HP'].max()
avg_torque = data['Torque'].mean()
max_torque = data['Torque'].max()
avg_braking_distance = data['Braking'].mean()
max_braking_distance = data['Braking'].min()
avg_skid_pad = data['Road Holding'].mean()
max_skid_pad = data['Road Holding'].max()
avg_fuel_economy = data['Fuel Economy'].mean()
max_fuel_economy = data['Fuel Economy'].max()

d = {
    'Brand': data['Brand'], 
    'Body': data['Body'], 
    'Year': data['Year'],
    'Price': data['Price'],
    'Model': data['Model']
}

hp_score_list = []
torque_score_list = []
braking_score_list = []
turning_score_list = []
fuel_economy_score_list = []

for index, row in data.iterrows():
    calculated_HP_score = calculate_score(row, avg_HP, max_HP, 'HP')
    hp_score_list.append(calculated_HP_score)
    calculated_torque_score = calculate_score(row, avg_torque, max_torque, 'Torque')
    torque_score_list.append(calculated_torque_score)
    calculated_braking_score = calculate_score(row, avg_braking_distance, max_braking_distance, 'Braking')
    braking_score_list.append(calculated_braking_score)
    calculated_turning_score = calculate_score(row, avg_skid_pad, max_skid_pad, 'Road Holding')
    turning_score_list.append(calculated_turning_score)
    fuel_economy_score = calculate_score(row, avg_fuel_economy, max_fuel_economy, 'Fuel Economy')
    fuel_economy_score_list.append(fuel_economy_score)

df = pd.DataFrame(data=d)

df['HP'] = hp_score_list
df['Torque'] =  torque_score_list
df['Braking'] = braking_score_list
df['RoadHolding'] = turning_score_list
df['FuelEconomy'] = fuel_economy_score_list

df.to_csv('data/model_vectors.csv')